<a href="https://colab.research.google.com/github/mitran27/AiUtils/blob/main/PytorchHooks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
from torch.nn import Module

In [31]:
class CLayer (Module):
  def  __init__(self):super().__init__()
  def forward(self,x,y):return x*y
class Sample(Module):
  def __init__(self):
    super().__init__()
    self.a = torch.tensor(2.0, requires_grad=True)
    self.b = torch.tensor(3.0, requires_grad=True)
    self.d = torch.tensor(4.0, requires_grad=True)
    self.c = CLayer()
    self.temp = []
    self.c.register_forward_hook(self.save_hooksO)
    self.c.register_forward_pre_hook(self.save_hooksI)
    self.a.register_hook(self.update_grad)
    self.c.register_backward_hook(self.view_grad) # returns all propertis of grad for the layer(inputs,output)

  def save_hooksI(self,module,inputs):
    self.temp.append(inputs)
  def save_hooksO(self,module,inputs,outputs):
     self.temp.append(outputs)
  def update_grad(self,grad):
    print(grad)
    self.temp.append(grad)
    return grad+2
  def view_grad(self,module,grad_in,grad_out):
    self.temp.append((grad_in,grad_out))



  def forward(self):
    c  =self.c(self.a,self.b)
    #c.register_hook(self.update_grad) returns only the grad for the operation
    e = c * self.d
    e.register_hook(self.update_grad)

    e.backward()







In [32]:
newnet = Sample()
newnet()
print("values in temp")
newnet.temp

tensor(1.)
tensor(36.)
values in temp


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1359: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


[(tensor(2., requires_grad=True), tensor(3., requires_grad=True)),
 tensor(6., grad_fn=<MulBackward0>),
 tensor(1.),
 ((tensor(36.), tensor(24.)), (tensor(12.),)),
 tensor(36.)]